In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import joblib as jb
from skimage import io
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

%pip install opencv-python



Note: you may need to restart the kernel to use updated packages.


In [2]:
#define image path
pos_path = "archive/1"
neg_path = "archive/0"

pos_image_listing = os.listdir(pos_path)
neg_image_listing = os.listdir(neg_path)

print('Number of positive images:', pos_image_listing)
print('Number of negative images:', neg_image_listing)

Number of positive images: ['0.png', '1.png', '10.png', '100.png', '101.png', '102.png', '103.png', '104.png', '105.png', '106.png', '107.png', '108.png', '109.png', '11.png', '110.png', '111.png', '112.png', '113.png', '114.png', '115.png', '116.png', '117.png', '118.png', '119.png', '12.png', '120.png', '121.png', '122.png', '123.png', '124.png', '125.png', '126.png', '127.png', '128.png', '129.png', '13.png', '130.png', '131.png', '132.png', '133.png', '134.png', '135.png', '136.png', '137.png', '138.png', '139.png', '14.png', '140.png', '141.png', '142.png', '143.png', '144.png', '145.png', '146.png', '147.png', '148.png', '149.png', '15.png', '150.png', '151.png', '152.png', '153.png', '154.png', '155.png', '156.png', '157.png', '158.png', '159.png', '16.png', '160.png', '161.png', '162.png', '163.png', '164.png', '165.png', '166.png', '167.png', '168.png', '169.png', '17.png', '170.png', '171.png', '172.png', '173.png', '174.png', '175.png', '176.png', '177.png', '178.png', '179.

In [3]:
#define parameters
orientation = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = 0.3

data=[]
labels=[]

#extract HOG features from positive images
for i in range(len(pos_image_listing)):
    image = cv2.imread(pos_path + "/" + pos_image_listing[i])
    image = cv2.resize(image, (64, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, hog_image = hog(gray, orientations=orientation, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)
    data.append(fd)
    labels.append(1)
for i in range(len(neg_image_listing)):
    image = cv2.imread(neg_path + "/" + neg_image_listing[i])
    image = cv2.resize(image, (64, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, hog_image = hog(gray, orientations=orientation, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)
    data.append(fd)
    labels.append(0)

In [5]:
le = LabelEncoder()
labels = le.fit_transform(labels)
print('Contructing training in spliting data')
data = np.array(data)
labels = np.array(labels)

#split data into training and testing
Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, labels, test_size=0.2, random_state=42)


data_arr = np.array(Data_train)
labels_arr = np.array(Data_test)

# Lưu mảng thành file văn bản
#np.savetxt('data.txt', data_arr)
#np.savetxt('labels.txt', labels_arr)

Contructing training in spliting data


In [6]:
#Train Linear SVM

model = LinearSVC(max_iter=10000, random_state=42, tol=1e-5, verbose=1)
model.fit(Data_train, Labels_train)
predictions = model.predict(Data_test)

print('Accuracy:', model.score(Data_test, Labels_test))

jb.dump(model, 'model.xml')


c:\Users\trank\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[LibLinear]

KeyboardInterrupt: 

In [4]:
def inside(r, q):
    rx, ry, rw, rh = r
    qx, qy, qw, qh = q
    return rx > qx and ry > qy and rx + rw < qx + qw and ry + rh < qy + qh
def draw_detections(img, rects, thickness = 1):
    for x, y, w, h in rects:
        # the HOG detector returns slightly larger rectangles than the real objects.
        shrink_ratio_w = 0.2
        shrink_ratio_h = 0.2
        pad_w = int(shrink_ratio_w * w)
        pad_h = int(shrink_ratio_h * h)
        
        # so we slightly shrink the rectangles to get a nicer output.
        pad_w, pad_h = int(0.15*w), int(0.05*h)
        cv2.rectangle(img, (x+pad_w, y+pad_h), (x+w-pad_w, y+h-pad_h), (0, 255, 0), thickness)
model = jb.load('model.xml')
hog = cv2.HOGDescriptor()
hog.setSVMDetector(model.coef_.ravel())

cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

while True:
    ret, frame = cam.read()
    found, w = hog.detectMultiScale(frame, winStride=(16, 16), padding=(32, 32), scale=1.05)
    draw_detections(frame, found)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

In [3]:
import cv2
help(cv2.HOGDescriptor_getDefaultPeopleDetector())


Help on ndarray object:

class ndarray(builtins.object)
 |  ndarray(shape, dtype=float, buffer=None, offset=0,
 |          strides=None, order=None)
 |
 |  An array object represents a multidimensional, homogeneous array
 |  of fixed-size items.  An associated data-type object describes the
 |  format of each element in the array (its byte-order, how many bytes it
 |  occupies in memory, whether it is an integer, a floating point number,
 |  or something else, etc.)
 |
 |  Arrays should be constructed using `array`, `zeros` or `empty` (refer
 |  to the See Also section below).  The parameters given here refer to
 |  a low-level method (`ndarray(...)`) for instantiating an array.
 |
 |  For more information, refer to the `numpy` module and examine the
 |  methods and attributes of an array.
 |
 |  Parameters
 |  ----------
 |  (for the __new__ method; see Notes below)
 |
 |  shape : tuple of ints
 |      Shape of created array.
 |  dtype : data-type, optional
 |      Any object that can